In [1]:
import entsoe_client as ec
from entsoe_client import ParameterTypes
import pandas as pd

In [7]:
# Instantiate client, parser and query.
client = ec.Client("6e68642c-8403-4caa-af31-bda40b8c67f6")
parser = ec.Parser()
query = ec.Queries.Generation.AggregatedGenerationPerType(
    in_Domain=ec.ParameterTypes.Area('DE_LU'),
    periodStart=202206010015,
    periodEnd=202206010230)

In [8]:
# Extract data.
response = client(query)
df = parser(response)
#df.to_csv("./test.csv")

In [9]:
consumption_mask = df["TimeSeries.outBiddingZone_Domain.mRID"].notna()
production = df[~consumption_mask][["quantity", "TimeSeries.MktPSRType.psrType"]]
#print(production.iloc[:,:10])
#print(df.iloc[:,:10])
#print(df.columns)
#print(consumption_mask)
#print(production)

In [10]:
production["GenerationType"] = production["TimeSeries.MktPSRType.psrType"].apply(
    lambda x: ParameterTypes.PsrType[x].value
)  # Use ParameterTypes to transform ENTSO-E Code into readable string.
production_by_type = pd.pivot_table(
    production, index=production.index, columns="GenerationType", values="quantity"
)

In [14]:
print(production.shape)

(144, 3)


In [11]:
print(production_by_type)
print(production_by_type.columns, production_by_type.shape)

GenerationType             Biomass  Fossil Brown coal / Lignite  Fossil Gas  \
2022-06-01 00:15:00+00:00   4411.0                      14696.0      7017.0   
2022-06-01 00:30:00+00:00   4404.0                      14723.0      7058.0   
2022-06-01 00:45:00+00:00   4388.0                      14634.0      7073.0   
2022-06-01 01:00:00+00:00   4370.0                      14614.0      7177.0   
2022-06-01 01:15:00+00:00   4379.0                      14577.0      7143.0   
2022-06-01 01:30:00+00:00   4391.0                      14598.0      7148.0   
2022-06-01 01:45:00+00:00   4397.0                      14603.0      7149.0   
2022-06-01 02:00:00+00:00   4409.0                      14600.0      7207.0   
2022-06-01 02:15:00+00:00   4414.0                      14600.0      7190.0   

GenerationType             Fossil Hard coal  Fossil Oil  Geothermal  \
2022-06-01 00:15:00+00:00            4328.0         8.0        25.0   
2022-06-01 00:30:00+00:00            4283.0         8.0        25.0